In [55]:
import featuretools as ft
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly as py

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report,roc_auc_score
import plotly.graph_objects as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from multiprocessing import Pool

import torch 
torch.manual_seed(0) 
import gpytorch 

In [3]:
data = pd.read_csv('train.csv')
data.dropna(inplace=True)
print(data.shape)
data.head(8)

(9000, 19)


,ID,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,Month,OperatingSystems,Browser,Region,TrafficType,VisitorType,Weekend,Revenue
0,0,2,208.000000,1,50.000000,30,1461.666667,0.000000,0.021250,0.0,0.0,1,1,7,0,12,2,0,0
1,1,1,0.000000,0,0.000000,1,0.000000,0.200000,0.200000,0.0,0.6,2,3,3,2,4,2,0,0
2,2,0,0.000000,1,1729.000000,19,1401.083333,0.020000,0.050000,0.0,0.0,9,1,10,0,12,2,0,0
3,3,0,0.000000,16,1210.397619,5,279.857143,0.003175,0.012764,0.0,0.0,1,1,10,0,11,2,0,0
4,4,1,7.000000,0,0.000000,50,2330.727381,0.023810,0.031399,0.0,0.0,8,5,10,6,12,2,0,0
5,5,7,245.333333,0,0.000000,5,256.666667,0.027273,0.051818,0.0,0.0,2,3,3,6,12,2,0,0
6,6,3,155.700000,0,0.000000,18,595.000000,0.000000,0.010526,0.0,0.0,8,3,3,0,4,1,0,0
7,7,6,48.625000,0,0.000000,14,211.470238,0.001190,0.039521,0.0,0.0,8,5,10,1,5,2,0,0


In [4]:
print(data.isnull().sum().sum())
data = pd.get_dummies(data)
le = LabelEncoder()
data['Revenue'] = le.fit_transform(data['Revenue'])
data['Revenue'].value_counts()

0


0    10408
1     1908
Name: Revenue, dtype: int64

In [5]:
x = data
x = x.drop(['Revenue'], axis = 1)
y = data['Revenue']
x = np.array(x).astype(np.float32)
y = np.array(y).astype(np.float32)

# from sklearn.preprocessing import StandardScaler
# x = StandardScaler().fit_transform(x)

from sklearn.preprocessing import PolynomialFeatures
x = PolynomialFeatures(3).fit_transform(x)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 0)
y_train[y_train==0] = -1
y_train[y_train==1] = 1
y_test[y_test==0] = -1
y_test[y_test==1] = 1

print("Shape of x_train :", x_train.shape)
print("Shape of y_train :", y_train.shape)
print("Shape of x_test :", x_test.shape)
print("Shape of y_test :", y_test.shape)

Shape of x_train : (8621, 4495)
Shape of y_train : (8621,)
Shape of x_test : (3695, 4495)
Shape of y_test : (3695,)


In [6]:
from sklearn.decomposition import PCA

pca = PCA(n_components=50)
pca.fit(x_train)
print(pca.explained_variance_ratio_)
print(np.sum(pca.explained_variance_ratio_))

[9.9994725e-01 3.4032397e-05 1.6829907e-05 1.7735119e-06 4.5245663e-08
 2.7771032e-08 7.7326838e-09 4.6951594e-09 4.0646388e-09 3.7409884e-09
 1.6170968e-09 1.2118884e-09 5.0365612e-10 3.3601710e-10 3.1669159e-10
 2.0677736e-10 1.0069006e-10 7.8262841e-11 4.3076223e-11 3.3719388e-11
 2.2855528e-11 1.4696926e-11 1.0883385e-11 6.2213594e-12 3.6997298e-12
 3.4655638e-12 2.9930231e-12 2.4525354e-12 2.0751244e-12 1.7279064e-12
 1.6352091e-12 1.3163522e-12 1.1637421e-12 1.0996902e-12 9.4876830e-13
 8.5623349e-13 7.6613444e-13 7.0682669e-13 6.0880559e-13 5.0128830e-13
 2.8195515e-13 2.7183014e-13 2.2745309e-13 2.1030951e-13 1.9929170e-13
 1.6221174e-13 1.4098869e-13 1.2016285e-13 1.1384037e-13 9.7252657e-14]
0.99999994


In [7]:
x_train, x_test = pca.transform(x_train), pca.transform(x_test)

In [8]:
print("Shape of x_train :", x_train.shape)
print("Shape of y_train :", y_train.shape)
print("Shape of x_test :", x_test.shape)
print("Shape of y_test :", y_test.shape)

Shape of x_train : (8621, 50)
Shape of y_train : (8621,)
Shape of x_test : (3695, 50)
Shape of y_test : (3695,)


In [9]:
from xgboost import XGBRegressor

sumpos = np.sum(y_train==1)
sumneg = np.sum(y_train==-1)
print(sumpos,sumneg)


1345 7276


In [ ]:
xgmodel = XGBRegressor(n_jobs=1,scale_pos_weight=sumneg/sumpos,eval_metric='auc')
# xgmodel = XGBRegressor()
print(xgmodel)
xgmodel.fit(x_train,y_train)
y_predxg = xgmodel.predict(x_test)
print("auc=",roc_auc_score(y_test,y_predxg))

In [192]:
x_train = np.load('x_train.npy',allow_pickle=True)
print("x_train.shape=",x_train.shape)
x_val = np.load('x_val.npy',allow_pickle=True)
print("x_val.shape=",x_val.shape)
y_train = np.load('y_train.npy',allow_pickle=True)
print("y_train.shape=",y_train.shape)
y_val = np.load('y_val.npy',allow_pickle=True)
print("y_val.shape=",y_val.shape)
x_test = np.load('x_test.npy',allow_pickle=True)
print("x_test.shape=",x_test.shape)

x_train.shape= (7200, 51)
x_val.shape= (1800, 51)
y_train.shape= (7200,)
y_val.shape= (1800,)
x_test.shape= (3330, 51)


In [194]:
data_train = pd.DataFrame(x_train)
data_train['Revenue'] = y_train

data_train.head()

,0,1,2,3,4,5,6,7,8,9,...,42,43,44,45,46,47,48,49,50,Revenue
0,2.0,52.000000,1.0,17.666666,18.0,1262.000000,0.0,0.009524,0.00000,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.000000,0.0,0.00,0.0
1,5.0,62.599998,0.0,0.000000,15.0,272.500000,0.0,0.011111,0.00000,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.000000,0.0,0.00,0.0
2,0.0,0.000000,0.0,0.000000,9.0,205.083328,0.0,0.044444,0.00000,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.000000,0.0,0.00,0.0
3,0.0,0.000000,0.0,0.000000,19.0,577.541687,0.0,0.011765,90.92852,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,52515.011719,0.0,997785.25,1.0
4,0.0,0.000000,0.0,0.000000,2.0,27.000000,0.0,0.050000,0.00000,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.000000,0.0,0.00,0.0


In [195]:
data_val = pd.DataFrame(x_val)
data_val['Revenue'] = y_val

data_val.head()

,0,1,2,3,4,5,6,7,8,9,...,42,43,44,45,46,47,48,49,50,Revenue
0,0.0,0.00,0.0,0.0,4.0,163.000000,0.050000,0.100000,0.00000,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.000000,0.200000,0.0,0.0
1,4.0,92.75,0.0,0.0,9.0,108.250000,0.000000,0.021429,0.00000,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.000000,0.000000,0.0,0.0
2,0.0,0.00,0.0,0.0,24.0,255.322220,0.025000,0.038889,0.00000,0.6,...,0.0,0.0,1.0,0.0,0.0,1.0,0.000000,0.600000,0.0,0.0
3,0.0,0.00,0.0,0.0,17.0,557.166687,0.023529,0.049510,0.00000,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.000000,0.400000,0.0,0.0
4,0.0,0.00,2.0,248.5,121.0,4218.074707,0.009985,0.026514,0.62451,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,2634.231445,1.208197,318742.0,0.0


In [196]:
data_test = pd.DataFrame(x_test)
data_test.head()

,0,1,2,3,4,5,6,7,8,9,...,41,42,43,44,45,46,47,48,49,50
0,0,0,0,0,141,2886.52,0.00431655,0.0187981,0,0.8,...,False,False,False,True,False,False,True,0,0.608633,0
1,0,0,0,0,8,64,0,0.05,0,0,...,False,False,True,False,False,False,True,0,0,0
2,0,0,0,0,15,139.833,0.0266667,0.0533333,0,0,...,False,False,False,True,False,False,True,0,0.4,0
3,0,0,5,497.833,26,851.667,0.00645161,0.0182796,0,0,...,False,False,True,False,False,True,False,0,0.167742,0
4,0,0,0,0,31,822.65,0.0206897,0.0208046,0,1,...,False,False,False,True,False,True,False,0,0.641379,0


In [25]:
data_train = pd.DataFrame(x_train,columns=['a', 'b', 'c', 'd','e','f','g','h','i','j','k','l','m','n','o','p','q','r']+[])
data_train

,a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r
0,0.622291,0.000000,0.000000,0.000000,0.000000,0.046809,0.011496,0.000000,0.013468,0.000000,0.0,0.888889,0.000000,0.833333,0.250,0.210526,1.0,0.0
1,0.727636,0.000000,0.000000,0.000000,0.000000,0.002837,0.000047,0.500000,0.750000,0.000000,0.0,0.222222,0.142857,0.833333,0.250,0.736842,1.0,0.0
2,0.442382,0.111111,0.009489,0.000000,0.000000,0.038298,0.019903,0.000000,0.044444,0.152444,0.0,0.888889,0.142857,0.833333,0.000,0.210526,0.5,0.0
3,0.358151,0.000000,0.000000,0.000000,0.000000,0.018440,0.002024,0.307692,0.461538,0.000000,0.0,0.111111,0.428571,0.916667,0.000,0.421053,1.0,0.0
4,0.634182,0.074074,0.009768,0.000000,0.000000,0.012766,0.002536,0.000000,0.090909,0.000000,0.0,0.666667,0.142857,0.833333,0.000,0.157895,1.0,0.0
5,0.965663,0.037037,0.002354,0.000000,0.000000,0.045390,0.010790,0.060606,0.149174,0.034923,0.0,1.000000,0.428571,0.250000,0.750,0.210526,1.0,1.0
6,0.168574,0.000000,0.000000,0.000000,0.000000,0.008511,0.000537,0.166667,0.250000,0.000000,0.6,0.222222,0.142857,0.833333,0.625,0.631579,1.0,0.0
7,0.417935,0.000000,0.000000,0.000000,0.000000,0.005674,0.001180,0.000000,0.250000,0.000000,0.0,0.111111,0.142857,0.833333,0.000,0.421053,1.0,1.0
8,0.264918,0.148148,0.024274,0.000000,0.000000,0.022695,0.004682,0.000000,0.027778,0.000000,0.0,1.000000,0.428571,0.250000,0.250,0.210526,1.0,1.0
9,0.719524,0.148148,0.006620,0.000000,0.000000,0.021277,0.005668,0.142857,0.196429,0.000000,0.0,1.000000,0.714286,0.833333,0.625,0.631579,1.0,0.0


In [26]:
data_val = pd.DataFrame(x_val,columns=['a', 'b', 'c', 'd','e','f','g','h','i','j','k','l','m','n','o','p','q','r'])
data_val

,a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r
0,0.491610,0.000000,0.000000,0.000000,0.000000,0.005674,0.002548,0.250000,0.500000,0.000000,0.0,1.000000,0.142857,0.833333,0.750,0.789474,1.0,0.0
1,0.191799,0.148148,0.027289,0.000000,0.000000,0.012766,0.001692,0.000000,0.107143,0.000000,0.0,0.888889,0.142857,0.833333,0.750,0.210526,0.5,0.0
2,0.877320,0.000000,0.000000,0.000000,0.000000,0.034043,0.003991,0.125000,0.194444,0.000000,0.6,0.222222,0.142857,0.833333,0.875,0.421053,1.0,0.0
3,0.842983,0.000000,0.000000,0.000000,0.000000,0.024113,0.008709,0.117647,0.247549,0.000000,0.0,0.111111,0.714286,0.833333,0.000,0.421053,1.0,0.0
4,0.220691,0.000000,0.000000,0.083333,0.097475,0.171631,0.065935,0.049925,0.132572,0.002169,0.0,1.000000,0.142857,0.833333,0.000,0.210526,1.0,0.0
5,0.335371,0.000000,0.000000,0.000000,0.000000,0.001418,0.000000,1.000000,1.000000,0.000000,0.0,0.222222,0.714286,0.833333,0.750,0.631579,1.0,0.0
6,0.163018,0.222222,0.064337,0.000000,0.000000,0.034043,0.027152,0.034483,0.075862,0.081666,0.0,0.888889,0.428571,0.250000,0.125,0.263158,1.0,1.0
7,0.885432,0.222222,0.095991,0.166667,0.036872,0.181560,0.079130,0.004274,0.089592,0.000000,0.0,0.000000,0.142857,0.583333,0.000,0.631579,1.0,0.0
8,0.017113,0.037037,0.022361,0.000000,0.000000,0.026950,0.019748,0.058824,0.117647,0.209556,0.6,0.222222,0.142857,0.833333,0.000,0.631579,1.0,0.0
9,0.257251,0.000000,0.000000,0.000000,0.000000,0.022695,0.008294,0.000000,0.062500,0.000000,0.0,0.888889,0.142857,0.583333,0.750,0.631579,1.0,1.0


In [41]:
data_test = pd.DataFrame(x_test,columns=['a', 'b', 'c', 'd','e','f','g','h','i','j','k','l','m','n','o','p','q','r'])
data_test.head()

,a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r
0,0.000000,0.0,0.0,0.000000,0.000000,0.264045,0.096312,0.021583,0.093990,0.0,0.8,0.222222,0.142857,0.818182,0.250,0.611111,1.0,0.0
1,0.000300,0.0,0.0,0.000000,0.000000,0.014981,0.002135,0.000000,0.250000,0.0,0.0,1.000000,0.142857,0.818182,0.750,0.555556,0.5,0.0
2,0.000601,0.0,0.0,0.000000,0.000000,0.028090,0.004666,0.133333,0.266667,0.0,0.0,0.111111,0.142857,0.818182,0.125,0.388889,1.0,0.0
3,0.000901,0.0,0.0,0.384615,0.221059,0.048689,0.028417,0.032258,0.091398,0.0,0.0,0.888889,0.142857,0.818182,0.000,0.166667,0.5,1.0
4,0.001202,0.0,0.0,0.000000,0.000000,0.058052,0.027449,0.103448,0.104023,0.0,1.0,0.222222,0.428571,0.181818,0.750,0.333333,1.0,1.0


In [28]:
data_train['Revenue'] = y_train
data_val['Revenue'] = y_val


In [178]:
nf_train = np.load('m11_train.npy')
nf_val = np.load('m11_val.npy')
nf_test = np.load('m11_pred.npy')
nf_train.shape

for i in range(6):
    if np.max(nf_train[:,i])-np.min(nf_train[:,i])!=0:
        nf_train[:,i] = (nf_train[:,i] - np.min(nf_train[:,i]))/(np.max(nf_train[:,i])-np.min(nf_train[:,i]))
    if np.max(nf_val[:,i])-np.min(nf_val[:,i])!=0:
        nf_val[:,i] = (nf_val[:,i] - np.min(nf_val[:,i]))/(np.max(nf_val[:,i])-np.min(nf_val[:,i]))
    if np.max(nf_test[:,i])-np.min(nf_test[:,i])!=0:
        nf_test[:,i] = (nf_test[:,i] - np.min(nf_test[:,i]))/(np.max(nf_test[:,i])-np.min(nf_test[:,i]))

In [179]:
nfnames = [ 'nf1', 'nf2', 'nf3', 'nf4', 'nf5', 'nf6' ]
for i in range(len(nfnames)):
    data_train[nfnames[i]] = nf_train[:,i]
    data_val[nfnames[i]] = nf_val[:,i]
    data_test[nfnames[i]] = nf_test[:,i]

In [180]:
data_train.shape

(6750, 31)

In [181]:
data_test.head()

,a,b,c,d,e,f,g,h,i,j,...,p3,p4,p5,p6,nf1,nf2,nf3,nf4,nf5,nf6
0,0.000000,0.0,0.0,0.000000,0.000000,0.264045,0.096312,0.021583,0.093990,0.0,...,0.0,0.008081,0.009436,0.011018,0.180746,0.440060,0.313492,0.483039,0.697624,0.721052
1,0.000300,0.0,0.0,0.000000,0.000000,0.014981,0.002135,0.000000,0.250000,0.0,...,0.0,0.333025,0.007144,0.000153,0.831529,0.242380,0.442926,0.474886,0.771821,0.542855
2,0.000601,0.0,0.0,0.000000,0.000000,0.028090,0.004666,0.133333,0.266667,0.0,...,0.0,0.001170,0.000308,0.000081,0.424979,0.318359,0.324441,0.473533,0.503143,0.607526
3,0.000901,0.0,0.0,0.384615,0.221059,0.048689,0.028417,0.032258,0.091398,0.0,...,0.0,0.257246,0.019229,0.001437,0.254025,0.821518,0.488877,0.443671,0.030169,0.524843
4,0.001202,0.0,0.0,0.000000,0.000000,0.058052,0.027449,0.103448,0.104023,0.0,...,0.0,0.008081,0.002272,0.000639,0.616427,0.505833,0.033288,0.526576,0.287183,0.412947


In [182]:
data_val.shape

(2250, 31)

In [39]:
data_val.head()

,a,b,c,d,e,f,g,h,i,j,...,p,q,r,Revenue,nf1,nf2,nf3,nf4,nf5,nf6
0,0.491610,0.000000,0.000000,0.000000,0.000000,0.005674,0.002548,0.250000,0.500000,0.000000,...,0.789474,1.0,0.0,0.0,0.371227,0.320151,0.506279,0.498063,0.327530,0.614014
1,0.191799,0.148148,0.027289,0.000000,0.000000,0.012766,0.001692,0.000000,0.107143,0.000000,...,0.210526,0.5,0.0,0.0,0.180301,0.286668,0.529618,0.625289,0.042798,0.579029
2,0.877320,0.000000,0.000000,0.000000,0.000000,0.034043,0.003991,0.125000,0.194444,0.000000,...,0.421053,1.0,0.0,0.0,0.356892,0.897670,0.670437,0.696746,0.551348,0.478374
3,0.842983,0.000000,0.000000,0.000000,0.000000,0.024113,0.008709,0.117647,0.247549,0.000000,...,0.421053,1.0,0.0,0.0,0.825339,0.563374,0.747653,0.693697,0.392333,0.592646
4,0.220691,0.000000,0.000000,0.083333,0.097475,0.171631,0.065935,0.049925,0.132572,0.002169,...,0.210526,1.0,0.0,0.0,0.763797,0.355579,0.398192,0.726296,0.161980,0.459765


In [183]:
data_test.shape

(3330, 30)

In [184]:
data_test.head()

,a,b,c,d,e,f,g,h,i,j,...,p3,p4,p5,p6,nf1,nf2,nf3,nf4,nf5,nf6
0,0.000000,0.0,0.0,0.000000,0.000000,0.264045,0.096312,0.021583,0.093990,0.0,...,0.0,0.008081,0.009436,0.011018,0.180746,0.440060,0.313492,0.483039,0.697624,0.721052
1,0.000300,0.0,0.0,0.000000,0.000000,0.014981,0.002135,0.000000,0.250000,0.0,...,0.0,0.333025,0.007144,0.000153,0.831529,0.242380,0.442926,0.474886,0.771821,0.542855
2,0.000601,0.0,0.0,0.000000,0.000000,0.028090,0.004666,0.133333,0.266667,0.0,...,0.0,0.001170,0.000308,0.000081,0.424979,0.318359,0.324441,0.473533,0.503143,0.607526
3,0.000901,0.0,0.0,0.384615,0.221059,0.048689,0.028417,0.032258,0.091398,0.0,...,0.0,0.257246,0.019229,0.001437,0.254025,0.821518,0.488877,0.443671,0.030169,0.524843
4,0.001202,0.0,0.0,0.000000,0.000000,0.058052,0.027449,0.103448,0.104023,0.0,...,0.0,0.008081,0.002272,0.000639,0.616427,0.505833,0.033288,0.526576,0.287183,0.412947


In [37]:
data_train.to_csv('data_train_wnf.csv')
data_val.to_csv('data_val_wnf.csv')
data_test.to_csv('data_test_wnf.csv')

In [ ]:
# data_train = pd.read_csv()

In [159]:
orig_feats = ['a', 'b', 'c', 'd','e','f','g','h','i','j','k','l','m','n','o','p','q','r']
x_train = np.array(data_train.loc[:,orig_feats])
y_train = np.array(data_train.loc[:,'Revenue'])
x_val = np.array(data_val.loc[:,orig_feats])
y_val = np.array(data_val.loc[:,'Revenue'])
x_test = np.array(data_test.loc[:,orig_feats])
print(x_train.shape,y_train.shape)


(6750, 18) (6750,)


In [160]:
from sklearn.preprocessing import PolynomialFeatures

# poly = PolynomialFeatures(3)
# x_train = poly.fit_transform(x_train)
print(x_train.shape)

(6750, 18)


In [161]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=300,max_features=None,class_weight='balanced')
rf.fit(x_train, y_train)

print ("Features sorted by their score:")
print (sorted(zip(map(lambda x: round(x, 4), rf.feature_importances_), orig_feats+nfnames), 
             reverse=True))

yh_val = rf.predict_proba(x_val)[:,1]
print(yh_val)
print(y_val)
roc_auc_score(y_val,yh_val)

Features sorted by their score:
[(0.493, 'j'), (0.0809, 'l'), (0.0561, 'i'), (0.0559, 'f'), (0.0559, 'a'), (0.0532, 'g'), (0.0393, 'h'), (0.036, 'c'), (0.0243, 'b'), (0.0227, 'p'), (0.0173, 'e'), (0.0167, 'o'), (0.0136, 'd'), (0.0104, 'n'), (0.0104, 'm'), (0.0053, 'r'), (0.0052, 'q'), (0.0038, 'k')]
[0.01       0.19666667 0.         ... 0.20666667 0.         0.15333333]
[0. 0. 0. ... 0. 0. 0.]


0.9222559599645458

In [162]:
most_imp_feats = [ 'j','l','f']
x_train_imp = np.log(np.array(data_train.loc[:,most_imp_feats])+1)
x_val_imp = np.log(np.array(data_val.loc[:,most_imp_feats])+1)
x_test_imp = np.log(np.array(data_test.loc[:,most_imp_feats])+1)
poly = PolynomialFeatures(3)
x_train_imp = poly.fit_transform(x_train_imp)
x_val_imp = poly.transform(x_val_imp)
x_test_imp = poly.transform(x_test_imp)
print(x_train_imp.shape)



(6750, 20)


In [163]:
x_train = np.concatenate((x_train,x_train_imp),axis=1)
x_val = np.concatenate((x_val,x_val_imp),axis=1)
x_test = np.concatenate((x_test,x_test_imp),axis=1)
x_train.shape

(6750, 38)

In [164]:
rf = RandomForestClassifier(n_estimators=500,max_features=None,class_weight='balanced')
rf.fit(x_train, y_train)

print ("Features sorted by their score:")
print (rf.feature_importances_)

yh_val = rf.predict_proba(x_val)[:,1]
# print(yh_val)
# print(y_val)
roc_auc_score(y_val,yh_val)

Features sorted by their score:
[0.04779203 0.02152027 0.03180961 0.01225637 0.01584909 0.00578015
 0.0414034  0.03432468 0.04809969 0.07611601 0.00328572 0.01796118
 0.00886577 0.0087137  0.01423452 0.01968106 0.00455251 0.00461495
 0.         0.07327526 0.0192239  0.00582322 0.07027681 0.00997826
 0.01790582 0.01724986 0.02056515 0.00630084 0.00289843 0.05544243
 0.19317447 0.00578764 0.00673415 0.01131552 0.0173712  0.0252322
 0.01872192 0.00586221]


0.9252013536529244

In [165]:
rf.feature_importances_

array([0.04779203, 0.02152027, 0.03180961, 0.01225637, 0.01584909,
       0.00578015, 0.0414034 , 0.03432468, 0.04809969, 0.07611601,
       0.00328572, 0.01796118, 0.00886577, 0.0087137 , 0.01423452,
       0.01968106, 0.00455251, 0.00461495, 0.        , 0.07327526,
       0.0192239 , 0.00582322, 0.07027681, 0.00997826, 0.01790582,
       0.01724986, 0.02056515, 0.00630084, 0.00289843, 0.05544243,
       0.19317447, 0.00578764, 0.00673415, 0.01131552, 0.0173712 ,
       0.0252322 , 0.01872192, 0.00586221])

In [166]:
ind_new = np.logical_and((rf.feature_importances_>0.01),np.arange(38)>27)

In [168]:
x_train_poly = x_train[:,ind_new]
x_val_poly = x_val[:,ind_new]
x_test_poly = x_test[:,ind_new]
x_train_poly.shape

(6750, 6)

In [169]:
pnames = [ 'p1' ,'p2' , 'p3' , 'p4' , 'p5', 'p6' ]
for i in range(6):
    data_train[pnames[i]] = x_train_poly[:,i]
    data_val[pnames[i]] = x_val_poly[:,i]
    data_test[pnames[i]] = x_test_poly[:,i]

In [197]:
data_train.to_csv('data_train_new.csv')
data_val.to_csv('data_val_new.csv')
data_test.to_csv('data_test_new.csv')

In [186]:
data_test.shape

(3330, 30)

In [189]:
list(data_val)

['a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 'Revenue',
 'nf1',
 'nf2',
 'nf3',
 'nf4',
 'nf5',
 'nf6',
 'p1',
 'p2',
 'p3',
 'p4',
 'p5',
 'p6']

In [190]:
list(data_test)

['a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 'p1',
 'p2',
 'p3',
 'p4',
 'p5',
 'p6',
 'nf1',
 'nf2',
 'nf3',
 'nf4',
 'nf5',
 'nf6']